# BuRNN tutorial: Preparing of the training dataset and NN model training

In this tutorial we will use Methanol in water as a model system. In context of BuRNN, we will have Methanol in the inner region and two solvation shells of water molecules will be the buffer region. The rest of the box will be the outer region. The tutorial will be focused on the training dataset generation and the neural network (NN) model training.
## Training dataset preparation
Starting point for generating of the database of the QM structures for NN model training are snapshots from MD simulation, as was described in the original paper. QM region of the system (inner + buffer region) is extracted by GROMOS program filter from every saved snapshot of the initial MD simulation. Filter program produces one output file which contains cooradinates of all extracted QM regions (for all the snapshots). With this file we start in this tutorial. Firstly, we will extract the coorordinates of individual QM regions into separated GROMOS cnf files (one file per snapshot). Then we will have everythink prepared for QM calculations.

We will use semi-empirical program [MOPAC](http://openmopac.net/manual/index.html) for QM calculations in this tutorial. In real practise the choice of the QM software is done by the user. However, automation of QM calculations is necessary to prepare training dataset in the reasonable time. In our case we will use in-house made python modules gromos and mopac (Do not confound with original programs [GROMOS](https://www.gromos.net/) and [MOPAC](http://openmopac.net/manual/index.html)) for such an automation.

QM calculations for all the snapshots generated by MD can take some time. Therefore, in this tutorial, we will perform QM calculations only for two MD snapshots to show the training dataset preparation procedure. In the end we will provide you models trained on our complete dataset for running the BuRNN simulation in the next step of the tutorial.


In [1]:
# in-house written modules for MOPAC QM calculations automation.
import mopac
import gromos
import additional_spk_utils

# standard python modules
from glob import glob
import os

In [2]:
cwd = os.getcwd() # current working directory

The first step in our tutorial is to separate QM regions of the intial MD snapshots (from filter_output_example.cnf) into individual cnf files. Class gromos.Extract_Buffer_Pls_Inner is used to do it. Resulting cnf files will be stored in buffer_pls_inner_region_configurations directory. The files will contain coordinates of the QM regions for individual MD snapshots (produced by GROMOS program filter).

The class gromos.Extract_Buffer_Pls_Inner takes the tuple of GROMOS filter output files (in our case it will have only one item, filter_output_example.cnf) as an argument and by calling method extract() QM regions are separated into individual cnf files.

In [3]:
filter_output_path = os.path.join('.', 'filter_output_example.cnf')
gromos.Extract_Buffer_Pls_Inner(filtered_cnfs=(filter_output_path,)).extract()

In [4]:
# list of individual cnf files
cnf_files_path = os.path.join('.', 'buffer_pls_inner_region_configurations')
cnf_files = sorted(glob(os.path.join(cnf_files_path, '*.cnf')))
cnf_files

['./buffer_pls_inner_region_configurations/buffer_pls_inner_00000.cnf',
 './buffer_pls_inner_region_configurations/buffer_pls_inner_00001.cnf']

We created two separated cnf files (for two MD snapshots) in the previous step. The files contain QM regions (inner + buffer regions) coordinates for the given snapshots. Now we are ready for QM calculations. We have to consider that configurations (snapshots) calculated by MD are probably not the ideal ones from the point of view of QM chemistry. Thus QM energy minimization should be perform to reach more favoured conformations. Moreover, one has to consider that training dataset should represent sufficient part of the conformational space of the given system. Therefore we will include all the energy minimization steps into the training dataset.

We will use mopac.Mopac_Minimization_Calculation class for the automation of MOPAC energy minimization. The class takes the list of paths to the cnf files as an argument (cnf_files). User can also specify whether buffer region should be minimized as well (argument freeze_buffer, in our case false -> buffer region will be minimized). MOPAC minimization will be performed for all the snapshots in the cnf_files argument. The results will be stored in MOPAC_results/buffer_pls_inner/aux_out directory.

In [5]:
# NOTE: the program will remove the previously created folder MOPAC_results (if exists). 
#It is recommended to rename the previously created folder if you would like to save the previous results.
mopac.Mopac_Minimization_Calculation(cnf_files, freeze_buffer=False).run_calculation()

In [6]:
# list of the MOPAC output files (aux files)
aux_files = sorted(glob(os.path.join(cwd, 'MOPAC_results', 'buffer_pls_inner', 'aux_out', '*.aux')))
aux_files

['/pool/radekc/CD_lab_project/gromos_tutorial/gromos_tutorial_livecoms/tutorial_files/t_06/train_dataset_tutorial/MOPAC_results/buffer_pls_inner/aux_out/buffer_pls_inner_00000.aux',
 '/pool/radekc/CD_lab_project/gromos_tutorial/gromos_tutorial_livecoms/tutorial_files/t_06/train_dataset_tutorial/MOPAC_results/buffer_pls_inner/aux_out/buffer_pls_inner_00001.aux']

We have produced 2 MOPAC minimization output files in the previous step. Now we have to run the single point (SP) MOPAC calculation for every minimization step. Moreover, in BuRNN approach we predict interaction energies between inner and buffer regions (see original paper for details). Thus, we have to extract individual minimization steps from the output files and then run SP calculation for the inner + buffer region (the whole QM region) and for buffer region separately. 
This is handled by mopac.Mopac_1scf_Calculation_Aux_In class which takes the following arguments:
- aux_files = list of paths to the MOPAC minimization output files
- inner_region_size = number of atoms in the inner region (it is used to extract the buffer region for the separate SP calculation)

Besides that proper geometry of SPC water molecules has to be preserved for the following BuRNN simulation in GROMOS. This step is included in 
mopac.Mopac_1scf_Calculation_Aux_In class (it is done by gromos.ReShake class). It requires additional arguments:
- topo_path = path to the topology file of the given system
- gromos_bin = path to the GROMOS md program
- imd_example = path to the initial imd file (will be modified during the process)
- mk_example = path to the initial mk_script arg file (will be modified during the process)
- mk_lib = path to the mk_script library

The output of the Mopac_1scf_Calculation_Aux_In class will be two SP MOPAC calculation files for every minimization step (one file for the inner + buffer region, one for buffer region separately). The output files will be stored in:
- MOPAC_results/minimization/buffer_pls_inner/aux_out for inner + buffer regions
- MOPAC_results/minimization/buffer/aux_out for buffer regions

In [7]:
# SP mopac calculations with reshake of water molecules
# NOTE: the program will remove the previously created folder MOPAC_results/minimization (if exists). 
#It is recommended to rename the previously created folder if you would like to save the previous results.
imd = os.path.join('/','home', 'radekc', 'input_examples', 'reshake', 'reshake.imd')
mk_script_example = os.path.join('/', 'home', 'radekc', 'input_examples', 'reshake', 'mk_reshake.arg')
lib = os.path.join('/', 'home', 'radekc', 'GROMOS', 'mk_script_libs', 'mk_script_spk_local.lib')
gromos_bin = os.path.join('/', 'home', 'radekc', 'GROMOS', 'GROMOS', 'gromosXX', 'gromosXX','BUILD', 'bin', 'md') 
mopac.Mopac_1scf_Calculation_Aux_In(aux_files, 
                                    inner_region_size=6, 
                                    topo_path='../../../../../topo/meoh_54a7.top',
                                    gromos_bin=gromos_bin,
                                    imd_example=imd,
                                    mk_example=mk_script_example,
                                    mk_lib=lib).run_calculation()

We finished QM calculations in previous step. Note that by MOPAC energy minimization we increased the size of the dataset from 2 to 860. It demonstrates the potential of energy minimization to significantly incerase the size of training dataset. On the other hand we have to consider that energy minimization can produce a lot of very similar structures, especially in the end of the minimization process. The latter means that clustering of the training structures can be recommended before the model training (see original paper).
In the following part of the tutorial we will build ASE database from our training snapshots, then the database will be used for NN model training. For that purpose we will use another in-house made module additional_spk_utils. Firstly, we will need additional_spk_utils.Build_AseDb_From_Mopac_Aux class to create an ASE database. The class takes the following arguments:
- complex_path = path to the mopac output files with the whole QM regions (inner + buffer regions, in our case MOPAC_results/minimization/buffer_pls_inner/aux_out)
- buffer_path = path to the mopac output files with the buffer regions (in our case MOPAC_results/minimization/buffer/aux_out)
- inner_region_size = number of atoms in the inner region
- db_name = name of the resulting ASE dataset
- db_properties = the properties which will be stored in the database (in our case: complex energy, buffer energy, energy, forces)
- metadata = metadata
- reference_energies = reference energies (energies in vacuum) for all the components in the QM region.

Here it is good time to describe the last argument reference_energies more in detail. As was mentioned above, in the BuRNN approach we use NN model to predict interaction energy between inner and buffer region in stead of the absolute energy difference (see original paper for more details). The reference energies are used to calculate the interaction energy. Now consider the example. We have inner + buffer region composed of Methanol (inner region) and 15 water molecules (buffer region). Interaction energy is calculate in the following way:
- The absolute energies for inner + buffer regions and buffer regions were calculated by MOPAC.
- The first number in reference_energies argument coresponds to the energy of Methanol in vacuum, whereas the second represents the energy of one water molecule in vacuum (both were calculated by MOPAC).
- Firstly we sum the energies of the individual components of both regions (inner + buffer and buffer) in vacuum.
- for our example the summation is done in the following way:
    - inner + buffer region = reference energy of Methanol + 15 * reference energy of water molecule
    - buffer region = 15 * reference energy of water molecule
- The energies calculetad in the previous step are then subtracted from the original energies calculated MOPAC.

What we obtain in last step? We obtained the interaction energies. For the inner + buffer region we got the interaction energies between Methanol and between water molecules themselves. In case of buffer region we got interaction energy between water molecules only. By subtracting these two number we obtain the interaction energy between Methanol and water molecules (thus between inner and buffer region). This number is hidden under the name energy in db_properties argument. Similar procedure is done for the forces. The difference is that in the case of forces we are interested in atomic contributions. Therefore normalization of the forces is done by subtracting values for the inner + buffer region and buffer region. The values for the inner region are passed without any subtraction (they are not in the buffer region, so there is nothing to subtract). The subtracted values are hidden behind the name forces in db_properties argument.

class additional_spk_utils.Build_AseDb_From_Mopac_Aux also takes the argument metadata where one can describe the database. It is good to provide key information about the database to make it clear for other users or for you in the future.

In [20]:
# Example of dataset description
metadata = {'System' : 'MeOH in water',
            'num. of structures' : 860,
            'QM software' : 'MOPAC',
            'Energy minimization': 'Yes',
            'clustering' : 'No',
            'Energy units' : 'kcal/mol',
            'Force units' : 'kcal/mol/Angstrom',
            'distance units' : 'Angstrom'}

In [21]:
# building of ASE database
# NOTE: the program will remove the previous ASE database of the same name specified in db_name argument
additional_spk_utils.Build_AseDb_From_Mopac_Aux(complex_path=os.path.join('.', 'MOPAC_results', 'minimization', 'buffer_pls_inner', 'aux_out'),
                                                buffer_path=os.path.join('.', 'MOPAC_results', 'minimization', 'buffer', 'aux_out'),
                                                db_name='meoh_trial.db', 
                                                db_properties=['complex_energy', 'buffer_energy', 'energy', 'forces'], 
                                                inner_region_size=6, 
                                                reference_energies=[-48.9383958635117, -57.7996759075731],
                                                metadata=metadata).build_db()

In [22]:
# to check that the database was created
db = additional_spk_utils.Build_AseDb(load_existing_database=True, db_name='meoh_trial.db').create_db()
print(f'Number of structures: {len(db)}')
print(f'Interaction energy for the first structure: {db.__getitem__(0)["energy"][0]:.2f} kcal/mol')

Number of structures: 860
Interaction energy for the first structure: -19.37 kcal/mol


In [23]:
# show metadata
db.get_metadata()

{'System': 'MeOH in water',
 'num. of structures': 860,
 'QM software': 'MOPAC',
 'Energy minimization': 'Yes',
 'clustering': 'No',
 'Energy units': 'kcal/mol',
 'Force units': 'kcal/mol/Angstrom',
 'distance units': 'Angstrom'}

## Model trainig

Dataset of the training structures was generated in the previous parts. Now we can proceed to the training of the NN model (machine learned potential). Oue model will be based on the [SchNet](https://pubs.aip.org/aip/jcp/article-abstract/148/24/241722/962591/SchNet-A-deep-learning-architecture-for-molecules?redirectedFrom=fulltext) architecture. For the training of the model we will use [SchNetPack](https://pubs.acs.org/doi/10.1021/acs.jctc.8b00908) package. 

The [SchNet](https://pubs.aip.org/aip/jcp/article-abstract/148/24/241722/962591/SchNet-A-deep-learning-architecture-for-molecules?redirectedFrom=fulltext) model is a convolutional neural network (CNN) with a continuous filter. It is very similar to the common CNNs used in image recognition, for instance. In contrast to the images, molecules cannot be described by the discrete matrix of pixels and thus the continuous filter is applied instead of a discrete one. The SchNet model is composed of two NNs. The main one is responsible for the prediction of the given property itself (input is the vector of atomic numbers for the given structure). The second one generates the filter for the convolution (input: positions of the individual atoms of the given structure). The main NN is divided into three main blocks. The first is the embedding layer which creates the feature vectors for the individual atoms within the structure (therefore the whole structure is represented by the 2D matrix of shape(number of atom-wise features, number of atoms)). The second part of the model is the series of the interaction blocks. One interaction block contains one convolutional layer. This block is responsible for creating the representation of the system. The last part is the output module which predicts the given property of the structure (in our case energy). If you are interested in SchNet architecture in more detail see original [paper](https://pubs.aip.org/aip/jcp/article-abstract/148/24/241722/962591/SchNet-A-deep-learning-architecture-for-molecules?redirectedFrom=fulltext).

In practice, the SchNet model can be trained using python script spk_run.py provided with the SchNetPack package. Here we will use an additional bash script train.sh (is already prepared for you) to do so. The script will run spk_run.py with the specified arguments. spk_run.py takes the following arguments:
- positional arguments:
    - mode: str=train
    - architecture: str=schnet
    - dataset: str=custom 
    - datapath: str, path to the ASE database
    - modelpath: str, path to the model to be created
- optional arguments:
    - --help
    - --cuda = use Nvidia GPU for training
    - --parallel = parallel training on more GPUs
    - --seed = random seed for torch and numpy
    - --overwrite = remove previous model directory
    - --split_path = path to your own npz file with data split
    - --split = train, validation split; the rest of the dataset is used for testing
    - --max_epochs = maximum number of training epochs (default 5000)
    - --max_steps = maximum number of training steps (default None)
    - --lr = learning rate (default 0.0001)
    - --lr_decay = learning rate decay (default 0.8)
    - --lr_min = minimal learning rate (default 1e-06)
    - --lr_patience = epochs without improvement before reducing the learning rate (default 25)
    - --logger = logger (default csv)
    - --log_every_n_epochs = log metrics every given number of epochs (default 1)
    - --check_point_interval = store checkpoint every n epochs (default 1)
    - --keep_n_checkpoints = number of checkpoints that will be stored (default 3)
    - --environment_provider_device = It is recommended to use CPU (default cpu)
    - --features = number of atomwise features (default 128)
    - --interactions = number of interaction blocks (default 3)
    - --cutoff_function = default cosine
    - --num_gaussians = number of gaussians to expand distances (default 50)
    - --normalize_filter = normalize convolution filters by number of neighbors
    - --property = property to be predicted (default energy)
    - --cutoff = cutoff (default 10.0)
    - --batch_size = batch size (default 100)
    - --environment_provider = environment provider for the dataset (default simple)
    - --derivative = derivative of the property to be predicted (default None)
    - --negative_dr = multiply derivatives by -1 for training (when forces are provided instead of gradients, default False)
    - --force = name of force property in the dataset (alias for the derivative + negative_dr, default None)
    - --contributions = contributions of dataset property to be predicted (default None)
    - --stress = train on stress tensor if not None (default None)
    - --aggregation_mode = mode for aggregating atomix properties (default sum)
    - --output_module = select output module for the selected property (default atomwise)
    - --rho = tradeoff weights between property and derivative (default {})

In our case, we will train only a small model as an example of the model training process in SchNetPack. The complete models for the BuRNN simulation are provided in the directory "models". In our example, we will use our database (argument datapath). We need to specify in which directory the resulting model will be stored (argument modelpath). The training dataset has to be split into the training, validation and testing parts. In our case, we will use a random split (80 % training, 10 % validation and 10 % testing data). The sizes of training and validation data are specified in the argument split, respectively. The rest of the dataset is used for the final testing of the model. The model will be trained for 2 training epochs. we will use 32 atom-wise features and 1 interaction block to obtain a very small model which will be trained very quickly. Specify the necesseary arguments in the train.sh script and run it from the command line. The resulting model will be stored in the specified directory. Moreover, the directory contains log file (log.csv) and the file with the arguments used for the model training (args.json). The script will also run the spk_run.py in evaluation mode. The result of the evaluation will be stored in the model directory (evaluation.txt file). The evaluation will be done on the test data.

The hardest part of the model training process is the proper selection of the training hyperparameters. In practice, it is usually done by trial/error approach (at least partially). The general recommendation is to start with the default values of the SchNet architecture. If the resulting model does not fulfill your requirements, you can try to modify the individual hyperparameters. The hyperparameters number of atom-wise features (features) and number of interactions blocks (interactions) are the most important ones. These hyperparameters define the number of parameters of the model and thus its complexity. Therefore, one should start with modifying of those hyperparameters.